# Price prediction for Airbnb

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import math
import ast
import random
import numpy as np # linear algebra
from numpy import mean
from numpy import std
import matplotlib.pyplot as plt
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from pandas.plotting import scatter_matrix
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelBinarizer
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.compose import TransformedTargetRegressor
from sklearn.datasets import make_classification
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GroupKFold
from sklearn.model_selection import RepeatedKFold
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error, median_absolute_error, max_error
from sklearn.metrics import explained_variance_score
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import SCORERS


# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

#DATA_PATH = '/kaggle/input/airbnb-mexico-city'
DATA_PATH = './data'

import os
for dirname, _, filenames in os.walk(DATA_PATH):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

Importando arquivos

In [ ]:
df_calendar=pd.read_csv(DATA_PATH + '/calendar.csv')
df_listings=pd.read_csv(DATA_PATH + '/listings.csv')
df_reviews=pd.read_csv(DATA_PATH + '/reviews.csv')

In [ ]:
df_calendar.head()

In [ ]:
df_listings.columns

In [ ]:
df_listings['room_type'].value_counts()

### Seleção de colunas

Primeiramente faço um left join com todas as colunas entre calendar e listings.

In [ ]:
df_aux = pd.merge(
    df_calendar,
    df_listings,
    how="left",
    on=None,
    left_on='listing_id',
    right_on='id',
    left_index=False,
    right_index=False,
    sort=True,
    suffixes=("_x", "_y"),
    copy=True,
    indicator=False,
    validate=None,
)

Em seguida transformo preço e data em coluna numérica.

In [ ]:
df_aux['price_x'] = df_aux['price_x'].str.replace('$', '', regex = 'true').str.replace(',', '', regex = 'true')
df_aux['price_x'] = df_aux['price_x'].astype(float)

df_aux['date'] = pd.to_datetime(df_aux['date'], format='%Y-%m-%d', errors='ignore')
df_aux['ts'] = df_aux.date.values.astype(np.int64) // 10 ** 9
df_aux.drop(['date'],axis=1, inplace=True)

Calculando a matriz de correlação (Pearson) entre as colunas.

In [ ]:
corr_matrix = df_aux.corr()

Em seguida pode-se observar que as colunas accommodates, bedrooms e beds são as colunas numéricas que melhor se correlacionam com o preço.

In [ ]:
corr_matrix["price_x"].sort_values(ascending=False)

In [ ]:
del df_aux

In [ ]:
df = pd.merge(
    df_calendar[['listing_id', 'date', 'available', 'price']],
    df_listings[['id', 'latitude', 'longitude', 'property_type', 'room_type', 'accommodates', 'bedrooms', 'beds', 'amenities']],
    how="left",
    on=None,
    left_on='listing_id',
    right_on='id',
    left_index=False,
    right_index=False,
    sort=True,
    suffixes=("_x", "_y"),
    copy=True,
    indicator=False,
    validate=None,
)

In [ ]:
print(df_calendar.shape)
print(df.shape)

### Nulos

Checando valores nulos

In [ ]:
df.info()

Eliminando as linhas com qualquer valor nulo nos campos

In [ ]:
df.dropna(inplace=True)
df.shape

Transformando o preço de string para número

In [ ]:
df['price'] = df['price'].str.replace('$', '', regex = 'true').str.replace(',', '', regex = 'true')
df['price'] = df['price'].astype(float)

### Outliers

Analisando os preços, observa-se (pelo mínimo, máximo, média, mediana e desvio padrão) que existem outliers.

In [ ]:
print(df['price'].min(), df['price'].max(), df['price'].mean(), df['price'].median(), df['price'].std())

Removendo outliers de preços.

In [ ]:
fig, axs = plt.subplots(1, 1, figsize=(18, 6))
axs.hist(df['price'], bins=1000)
axs.set_title('Histograma de preços')

In [ ]:
df = df[df['price'].between(df['price'].quantile(.01), df['price'].quantile(.98))]

In [ ]:
fig, axs = plt.subplots(1, 1, figsize=(18, 6))
axs.hist(df['price'], bins=100)
axs.set_title('Histograma de preços')

Bem melhor agora.

In [ ]:
print(df['price'].min(), df['price'].max(), df['price'].mean(), df['price'].median(), df['price'].std())

Removendo coluna inútil id.

In [ ]:
df.drop(['id'],axis=1, inplace=True)

### Tratamentos sofisticados (amenities e date)

Tratando as cortesias (amenities)

In [ ]:
df_listings['amenities'].value_counts()

Funções para retornar um conjunto com todas as cortesias sem repetições, outra que retorna um dicionário com um valor numérico aleatório para cada cortesia e outra que retorna a soma dos valores das cortesias de um determinado registro. O objetivo é transformar cada lista de cortesias em um valor numérico único.

In [ ]:
amenities_dict = dict()

def getUniqueAmenities():
    rows = df_listings['amenities'].tolist()
    conj = set()
    for r in rows:
        conj = conj.union(set(ast.literal_eval(r)))
    return conj

def transformAmenities(set_str):
    lista = ast.literal_eval(set_str)
    return sum([amenities_dict[v] for v in lista])


In [ ]:
amenities_set = getUniqueAmenities()
for a in amenities_set:
    amenities_dict[a] = random.random()

Transformando os conjuntos de cortesias de cada registro em um valor numérico.

In [ ]:
df['amenities_ok'] = df['amenities'].apply(lambda x: transformAmenities(x))
df.drop(['amenities'], axis=1, inplace=True)

Transformando a data de string para seu respectivo timestamp, ou seja, um valor numérico único para cada data.

In [ ]:
df['date'] = pd.to_datetime(df['date'], format='%Y-%m-%d', errors='ignore')
df['ts'] = df.date.values.astype(np.int64) // 10 ** 9
df.drop(['date'],axis=1, inplace=True)

In [ ]:
corr_matrix = df.corr()
corr_matrix["price"].sort_values(ascending=False)

In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(18, 6))
axs[0].hist(df['latitude'], bins=100)
axs[1].hist(df['longitude'], bins=100)
print('Latitude e Longitude')

Removendo outliers de latitude e longitude

In [ ]:
df = df[df['latitude'].between(df['latitude'].quantile(.1), df['latitude'].quantile(.90))]

In [ ]:
df = df[df['longitude'].between(df['longitude'].quantile(.1), df['longitude'].quantile(.90))]

In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(18, 6))
axs[0].hist(df['latitude'], bins=100)
axs[1].hist(df['longitude'], bins=100)
print('Latitude e Longitude')

In [ ]:
corr_matrix = df.corr()
corr_matrix["price"].sort_values(ascending=False)

In [ ]:
df.describe()

In [ ]:
df.hist(bins=50, figsize=(20,15))
plt.show()

### Dados prontos

Agora que já temos somente colunas numéricas ou categóricas, vamos aplicar um pipeline para normalização e codificação:

In [ ]:
df.shape

In [ ]:
df_sample = df.sample(frac = 0.1)
df_sample.shape

Dividindo dados de treino e teste.

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(df_sample, df_sample['price'], test_size = .2, random_state = 0)

Aplicando pipeline de normalização e codificação, ou seja, transformando colunas numéricas em valores entre 0 e 1, e colunas categóricas em multiplas colunas com 0 e 1 de acordo a respectiva categoria.

In [ ]:
num_attribs = ['latitude', 'longitude', 'accommodates', 'bedrooms', 'beds', 'amenities_ok', 'ts']
cat_attribs = ['available','property_type', 'room_type']
pipeline_pre = ColumnTransformer([
    ('num', MinMaxScaler(), num_attribs),
    ('cat', OneHotEncoder(handle_unknown = "ignore"), cat_attribs),
])

X_train = pipeline_pre.fit_transform(X_train)
X_test = pipeline_pre.fit_transform(X_test)

Uma linha de base para comparação (média)

In [ ]:
y_test_mean = np.array([y_test.mean() for v in y_test])

print('RMSE: ', np.sqrt(mean_squared_error(y_test, y_test_mean)))
print('MAE: ', mean_absolute_error(y_test, y_test_mean))
print('MedAE: ', median_absolute_error(y_test, y_test_mean))
print('MaxError: ', max_error(y_test, y_test_mean))

### Regressão Linear

In [ ]:
lin_reg = LinearRegression()
lin_reg.fit(X_train, y_train)
y_pred = lin_reg.predict(X_test)

print('RMSE: ', np.sqrt(mean_squared_error(y_test, y_pred)))
print('MAE: ', mean_absolute_error(y_test, y_pred))
print('MedAE: ', median_absolute_error(y_test, y_pred))
print('MaxError: ', max_error(y_test, y_pred))

Aplicando uma Regressão Linear simples, vemos que o resultado foi ruim, com um grande erro. Talvez o modelo esteja sub-ajustado.

### Árvore de decisão

In [ ]:
dectree_reg = DecisionTreeRegressor()
dectree_reg.fit(X_train, y_train)
y_pred = dectree_reg.predict(X_test)

print('RMSE: ', np.sqrt(mean_squared_error(y_test, y_pred)))
print('MAE: ', mean_absolute_error(y_test, y_pred))
print('MedAE: ', median_absolute_error(y_test, y_pred))
print('MaxError: ', max_error(y_test, y_pred))

Aplicando uma Regressão com Árvore de Decisão, vemos que o resultado foi aceitável, porém com grandes chances de o modelo estar sobreajustado pois não houve validação com KFold.

### Validação cruzada com K Folds agrupados usando Regressão Linear, garantindo que cada apartamento estará em Folds diferentes

Primeiro crio 10 grupos baseado no listing_id para garantir que cada apartamento esteja no mesmo Fold

In [ ]:
df_sample['group'] = df_sample['listing_id'] % 10

In [ ]:
df_sample['group'].value_counts()

In [ ]:
group_kfold = GroupKFold(n_splits=10)
for train_index, test_index in group_kfold.split(df_sample, df_sample['price'], df_sample['group']):
    print("TRAIN:", train_index, "TEST:", test_index)

In [ ]:
#regr = TransformedTargetRegressor(regressor=LinearRegression(), func=np.log, inverse_func=np.exp)

# define the pipeline
steps = list()
steps.append(('pre', pipeline_pre))
steps.append(('model', LinearRegression()))
pipeline_lnrg = Pipeline(steps=steps)

scores = cross_val_score(pipeline_lnrg, df_sample, df_sample['price'],
                         groups=df_sample['group'], scoring='neg_mean_squared_error', 
                         cv=group_kfold, n_jobs=-1, error_score="raise")
print('RMSE = {}(+/- {})'.format(np.sqrt(-scores.mean()), np.sqrt(scores.std())))

### Validação cruzada com K Folds agrupados usando Árvore de Decisão, garantindo que cada apartamento estará em Folds diferentes

In [ ]:
#regr = TransformedTargetRegressor(regressor=DecisionTreeRegressor(), func=np.log, inverse_func=np.exp)

steps = list()
steps.append(('pre', pipeline_pre))
steps.append(('model', DecisionTreeRegressor()))
pipeline_dectree = Pipeline(steps=steps)
group_kfold = GroupKFold(n_splits=10)

scores = cross_val_score(pipeline_dectree, df_sample, df_sample['price'],
                         groups=df_sample['group'], scoring='neg_mean_squared_error', 
                         cv=group_kfold, n_jobs=-1, error_score="raise")
print('RMSE = {}(+/- {})'.format(np.sqrt(-scores.mean()), np.sqrt(scores.std())))

### Conclusão

Dados os resultados, vimos que a Regressão Linear foi um modelo melhor que a Árvore de decisão, apesar de um RMSE alto para ambos. Talves uma melhor engenharia de features possa melhorar esse modelo.